In [1]:
from torch.utils.data import Dataset
import torch
import os
from datasets import load_dataset
os.environ["TOKENIZERS_PARALLELISM"] = "false"
from transformers import AutoTokenizer

/root/vermind/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## pretrain dataset

In [2]:
ds = load_dataset("json",data_files="../dataset/pretrain_hq.jsonl",split="train",cache_dir="../.cache/pretrain_dataset")

In [3]:
ds[0]

{'text': '鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> 好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> 打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> 为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> 非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> 帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。<|im_end|> 回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> 识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。\n文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>'}

原始数据使用datasets库记载后的格式是 text + eos，

In [4]:
tokenizer = AutoTokenizer.from_pretrained("../vermind_tokenizer")

In [5]:
tokens = tokenizer(
            str(ds[0]["text"]),
            add_special_tokens=False,
            max_length= 512  - 2, # 因为后面需要手动添加bos，eos
            truncation=True # 超出长度截断
        ).input_ids
print(tokenizer.decode(tokens))
print(tokenizer.special_tokens_map)

鉴别一组中文文章的风格和特点，例如官方、口语、文言等。需要提供样例文章才能准确鉴别不同的风格和特点。<|im_end|> 好的，现在帮我查一下今天的天气怎么样?今天的天气依据地区而异。请问你需要我帮你查询哪个地区的天气呢？<|im_end|> 打开闹钟功能，定一个明天早上七点的闹钟。好的，我已经帮您打开闹钟功能，闹钟将在明天早上七点准时响起。<|im_end|> 为以下场景写一句话描述：一个孤独的老人坐在公园长椅上看着远处。一位孤独的老人坐在公园长椅上凝视远方。<|im_end|> 非常感谢你的回答。请告诉我，这些数据是关于什么主题的？这些数据是关于不同年龄段的男女人口比例分布的。<|im_end|> 帮我想一个有趣的标题。这个挺有趣的："如何成为一名成功的魔术师" 调皮的标题往往会吸引读者的注意力。<|im_end|> 回答一个问题，地球的半径是多少？地球的平均半径约为6371公里，这是地球自赤道到两极的距离的平均值。<|im_end|> 识别文本中的语气，并将其分类为喜悦、悲伤、惊异等。
文本：“今天是我的生日！”这个文本的语气是喜悦。<|im_end|>
{'bos_token': '<|im_start|>', 'eos_token': '<|im_end|>', 'unk_token': '<|endoftext|>', 'pad_token': '<|endoftext|>'}


> 可以看到一模一样，这个例子是没有截断的，感觉最后有时候没必要强行加一个eos啊，甚至说，这个bos有意义吗？？？

> ai的回答: 

| 特性 | 加上 BOS 和 EOS | 不加（错误示范） |
|------|-----------------|------------------|
| 训练目标 | 模型学会从零开始启动，并在说完后准确停止。 | 模型可能难以进入状态，且学会了“话唠”（无法停止）。 |
| 推理表现 | 用户提问 → 模型回答 → 模型输出 `<im_end>` | `im_end` |
| Attention | 首个 Token 关注 BOS，注意力分配正常。 | 首个 Token 无处关注，可能导致内部参数分布异常。 |

感觉主要是这个attn的问题


In [6]:
class PretrainDataset(Dataset):
    def __init__(self, data_path, tokenizer, max_length=512, cache_dir="../.cache/pretrain_dataset"):
        super().__init__()
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.cache_dir = cache_dir

        self.samples = load_dataset(
            "json",
            data_files=data_path,
            split="train",
            cache_dir=cache_dir,   
        )
        print(f"[PretrainDataset] HF cache dir: {self.cache_dir}")

    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, index):
        sample = self.samples[index]
        tokens = self.tokenizer(
            str(sample["text"]),
            add_special_tokens=False,
            max_length=self.max_length - 2, # 后面需要手动添加bos，eos
            truncation=True # 超出长度截断
        ).input_ids

        tokens = [self.tokenizer.bos_token_id] + tokens + [self.tokenizer.eos_token_id] # 加上 bos和eos
        input_ids = tokens + [self.tokenizer.pad_token_id] * (self.max_length - len(tokens)) # len(tokens) <= 512
        input_ids = torch.tensor(input_ids, dtype=torch.long) # 凡是索引，必须用 long（64位整数）类型

        labels = input_ids.clone() # 这里先不做shift
        labels[input_ids == self.tokenizer.pad_token_id] = -100 # 对于padding 的位置，label设置为-1

        return input_ids, labels

In [7]:
dataset = PretrainDataset(
    data_path="../dataset/pretrain_hq.jsonl",
    tokenizer=tokenizer,
    max_length=512,
)

[PretrainDataset] HF cache dir: ../.cache/pretrain_dataset
